In [1]:
import sys
import os
import pandas as pd
import numpy as np

import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

# Get the current working directory
# Get project root (folder above 'notebooks')
project_root = os.path.dirname(os.getcwd())

# Path to scripts folder
scripts_path = os.path.join(project_root, "scripts")
analysis_path = os.path.join(project_root, "analysis")
model_path = os.path.join(project_root, "model")
data_preparation_path = os.path.join(project_root, "data_preparation")
prediction_path = os.path.join(project_root, "prediction")

# Add to sys.path
sys.path.append(project_root)
sys.path.append(scripts_path)
sys.path.append(analysis_path)
sys.path.append(model_path)
sys.path.append(data_preparation_path)
sys.path.append(prediction_path)

import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [2]:
from data_preparation import preprocess_data
from model import build_severity_models,train_models
from prediction import evaluate_regression_model,evaluate_all_models,explain_model_predictions,calculate_risk_premium

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report
import xgboost as xgb
import shap
from typing import Tuple, Dict, Any, Optional

In [4]:
df_clean = pd.read_csv("../data/df_clean.csv")

C:\Users\hp\AppData\Local\Temp\ipykernel_24252\421350051.py:1: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  df_clean = pd.read_csv("../data/df_clean.csv")


In [5]:
# 1. Create the binary Claim Frequency target
df_clean['HasClaim'] = np.where(df_clean['TotalClaims'] > 0, 1, 0)

# 2. Create the Margin feature
df_clean['Margin'] = df_clean['TotalPremium'] - df_clean['TotalClaims']

# 3. Create the Claims_Only DataFrame for Severity calculation
claims_only_df = df_clean[df_clean['HasClaim'] == 1].copy()

In [6]:

float_cols = df_clean.select_dtypes(include=['float64']).columns
df_clean[float_cols] = df_clean[float_cols].astype('float32')

### Data Preparation and Split
We prepare the data for the two distinct modeling goals.

##### A. Claim Severity Model (Regression)
Goal: Predict TotalClaims amount, only for policies with claims.

In [7]:
X_train_sev, X_test_sev, y_train_sev, y_test_sev, preprocessor_sev = \
    preprocess_data(df_clean, target_col='TotalClaims', claims_only=True)

print("--- Claim Severity Data (Regression) ---")
print(f"Total training samples (policies with claims): {len(X_train_sev):,}")
print(f"Total test samples (policies with claims): {len(X_test_sev):,}")

--- Claim Severity Data (Regression) ---
Total training samples (policies with claims): 2,230
Total test samples (policies with claims): 558


##### B. Claim Probability Model (Classification)
Goal: Predict HasClaim (0 or 1), using the full dataset.

In [8]:
# NOTE: The preprocess_data function for classification needs an additional argument
# to handle the classifier models in model.py. (We'll assume classification model 
# definitions are added to model.py for this step)

# Re-defining required model building function to include classifiers
def build_claim_prob_models(preprocessor: Pipeline) -> Dict[str, Pipeline]:
    """
    Builds Claim Probability classification models (HasClaim).
    """
    models = {
        'RandomForestClassifier': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, max_depth=10, class_weight='balanced'),
        'XGBoostClassifier': xgb.XGBClassifier(objective='binary:logistic', n_estimators=100, random_state=42, n_jobs=-1, max_depth=5, eval_metric='logloss')
    }

    pipelines = {}
    for name, model in models.items():
        pipelines[name] = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('classifier', model)
        ])
    return pipelines

# Prepare data for probability model
X_train_prob, X_test_prob, y_train_prob, y_test_prob, preprocessor_prob = \
    preprocess_data(df_clean, target_col='HasClaim', claims_only=False)

print("\n--- Claim Probability Data (Classification) ---")
print(f"Total training samples: {len(X_train_prob):,}")
print(f"Claim Rate in Training: {y_train_prob.mean():.4f}")


--- Claim Probability Data (Classification) ---
Total training samples: 800,078
Claim Rate in Training: 0.0028


### Model Building and Training

##### A. Train Claim Severity Models (Regression)

In [9]:
# Build regression pipelines
pipelines_sev = build_severity_models(preprocessor_sev)
# Train models
trained_sev_models = train_models(pipelines_sev, X_train_sev, y_train_sev)


--- Starting Training on 2,230 samples ---
Training LinearRegression...
Training RandomForestRegressor...
Training XGBoostRegressor...


##### B. Train Claim Probability Models (Classification)

In [10]:
# Build classification pipelines
pipelines_prob = build_claim_prob_models(preprocessor_prob)
# Train models
trained_prob_models = train_models(pipelines_prob, X_train_prob, y_train_prob)


--- Starting Training on 800,078 samples ---
Training RandomForestClassifier...
Training XGBoostClassifier...


### Model Evaluation

##### A. Evaluate Claim Severity Models (Regression)

In [11]:
# Evaluate the regression models on the test set
sev_results = evaluate_all_models(trained_sev_models, X_test_sev, y_test_sev)
print("\n--- Claim Severity Model Evaluation (Regression) ---")
print(sev_results.to_markdown(floatfmt=".2f"))


--- Claim Severity Model Evaluation (Regression) ---
|                       |     RMSE |   R-squared |
|:----------------------|---------:|------------:|
| LinearRegression      | 36426.68 |        0.17 |
| RandomForestRegressor | 38374.88 |        0.08 |
| XGBoostRegressor      | 40078.62 |        0.00 |


## 📊 Model Evaluation Interpretation

### 1. Root Mean Squared Error (RMSE)

| Model                  | RMSE        | Meaning |
|------------------------|-------------|---------|
| **LinearRegression**   | 36,426.68   | On average, the model's prediction of a claim amount is off by R36,426.68. |
| **RandomForestRegressor** | 38,374.88 | Off by R38,374.88 on average. |
| **XGBoostRegressor**   | 40,078.62   | Off by R40,078.62 on average. |

**Interpretation:**  
RMSE measures the average magnitude of errors (difference between true and predicted claim amounts).  
It penalizes large errors heavily—so **lower RMSE is better**.

**Result:**  
- **Linear Regression** has the lowest RMSE, meaning it produced the most accurate predictions.  
- However, **an error above R36,000 is extremely high**, indicating the model still has poor predictive performance overall.

---

### 2. R-squared ($R^2$)

| Model                  | R-squared | Meaning |
|------------------------|-----------|---------|
| **LinearRegression**   | 0.17      | Explains 17% of the variance in claim severity. |
| **RandomForestRegressor** | 0.08   | Explains only 8% of the variance. |
| **XGBoostRegressor**   | 0.00      | Explains 0% of the variance (or less, rounded up). |

**Interpretation:**  
R-squared measures how much of the variability in the target variable is explained by the model.

- **1.0 → perfect model**  
- **0.0 → no explanatory power** (predicting the mean is just as good)

**Result:**  
- All models show **poor performance**, especially XGBoost.  
- Linear Regression explaining only **17%** of variance is weak.  
- XGBoost scoring **0.00** means it is learning nothing useful.

---


##### B. Evaluate Claim Probability Models

In [13]:
def evaluate_classification_model(model: Pipeline, X_test: pd.DataFrame, y_test: pd.Series) -> Dict[str, Any]:
    """Evaluates a classification model using Accuracy and Classification Report."""
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    
    return {'Accuracy': accuracy, 'F1-Score (Claim)': report['1']['f1-score'], 'Report': report}

# Evaluate all classification models
prob_results = {}
print("\n--- Claim Probability Model Evaluation (Classification) ---")
for name, model in trained_prob_models.items():
    metrics = evaluate_classification_model(model, X_test_prob, y_test_prob)
    prob_results[name] = metrics
    print(f"\nModel: {name}")
    print(f"Accuracy: {metrics['Accuracy']:.4f}")
    print(f"F1-Score (Claim, Class 1): {metrics['F1-Score (Claim)']:.4f}")


--- Claim Probability Model Evaluation (Classification) ---

Model: RandomForestClassifier
Accuracy: 0.9295
F1-Score (Claim, Class 1): 0.0721

Model: XGBoostClassifier
Accuracy: 0.9972
F1-Score (Claim, Class 1): 0.0000


## 📊 Claim Probability Model Evaluation Interpretation

### 1. The Core Problem: Class Imbalance
Insurance datasets are **heavily imbalanced** — most policies do **not** result in a claim (Class 0), and very few do (Class 1).

If claim frequency is around **1%–2%**, then a model that always predicts **NO CLAIM** would still achieve:

- **98%–99% accuracy**

👉 This means **Accuracy is a misleading metric** in this context.

---

### 2. Analysis of Metrics

| Model         | Accuracy | F1-Score (Class 1 — Claim) | Interpretation |
|---------------|----------|-----------------------------|----------------|
| **RandomForest** | 0.9295 | 0.0721 | Correctly classifies most samples overall, but performs very poorly at detecting claims. |
| **XGBoost**      | 0.9972 | 0.0000 | Nearly perfect accuracy because it predicts **NO CLAIM** almost every time. Completely fails to detect claims. |

---

### A. Accuracy (A Deceptive Metric)

- **XGBoost Accuracy = 0.9972**
- The model has simply “learned” that predicting **0 = No Claim** is almost always correct.
- High accuracy confirms **class imbalance**, not model quality.

👉 **Conclusion:** Accuracy is **not useful** in imbalanced insurance data.

---

### B. F1-Score (The Critical Metric)

The F1-score is the harmonic mean of Precision and Recall and is the **most important metric** for evaluating performance on the minority class (Claim / Class 1).

- **XGBoost F1 = 0.0000**  
  - Zero recall or zero precision.  
  - Found **no actual claims**, or all predicted claims were incorrect.  
  - The model **completely failed**.

- **RandomForest F1 = 0.0721**  
  - Slightly better, but still extremely poor.  
  - Barely better than random guessing at identifying actual claims.

---

## 🛑 Conclusion for AlphaCare's Pricing Framework

The **Claim Probability model is currently non-functional.**

The intended Advanced Pricing Framework is:

$$
\text{Premium} = \text{Predicted Probability of Claim} \times \text{Predicted Claim Severity} + \dots
$$

But both components are weak:

- **Predicted Claim Severity:**  
  - \( R^2 = 0.17 \) → Very weak explanatory power.

- **Predicted Claim Probability:**  
  - F1 ≈ **0.07** → Fails to detect claims reliably.

👉 **Result:**  
The current models are **not reliable enough** to support a risk-based premium calculation.

---


In [17]:
def explain_model_predictions2(model_pipeline: Pipeline, X_test: pd.DataFrame) -> Tuple[np.ndarray, pd.Index]:
    """
    Uses SHAP to analyze feature importance for the best performing model.
    Includes robust data conversion using Pandas to handle stubborn string-to-float errors.
    """
    # Isolate the regressor (the last step in the pipeline)
    model = model_pipeline.named_steps['regressor']
    
    # 1. Get preprocessed data for SHAP Explainer
    X_test_preprocessed = model_pipeline.named_steps['preprocessor'].transform(X_test)
    
    # Get feature names after one-hot encoding
    feature_names = model_pipeline.named_steps['preprocessor'].get_feature_names_out()
    
    # --- CRITICAL FIX: Robust Conversion via Pandas ---
    
    # 1a. Convert sparse output to dense array
    if hasattr(X_test_preprocessed, 'toarray'):
        X_test_dense = X_test_preprocessed.toarray()
    else:
        X_test_dense = X_test_preprocessed
        
    # 1b. Convert to a DataFrame to leverage pd.to_numeric for robust cleaning
    X_test_df_clean = pd.DataFrame(X_test_dense, columns=feature_names)
    
    # 1c. Apply pd.to_numeric across all columns, forcing errors (like '[...]' strings) to NaN
    # Then, we fill these NaNs with 0 (or a small median) before SHAP analysis.
    for col in X_test_df_clean.columns:
        X_test_df_clean[col] = pd.to_numeric(X_test_df_clean[col], errors='coerce')
        
    # Replace any resulting NaN values (from the conversion failure) with 0 for SHAP.
    # Using 0 is acceptable as SHAP will calculate the impact of that imputation.
    X_test_df_clean = X_test_df_clean.fillna(0)
    
    # Final matrix for SHAP
    X_test_numerical = X_test_df_clean.values
    # --- END CRITICAL FIX ---
    
    # 2. Initialize Explainer (TreeExplainer is efficient for XGBoost)
    explainer = shap.TreeExplainer(model)
    
    # 3. Calculate SHAP values
    shap_values = explainer.shap_values(X_test_numerical)
    
    return shap_values, feature_names

In [23]:
# Assuming XGBoostRegressor was the best severity model
best_sev_model = trained_sev_models['XGBoostRegressor']

# Run SHAP analysis (requires the model and preprocessed test data)
# NOTE: This step is complex and may fail in non-notebook environments.
# We will execute the function and print the top feature names.

try:
    shap_values_sev, feature_names_sev = explain_model_predictions2(best_sev_model, X_test_sev)

    # Calculate mean absolute SHAP value for overall importance
    mean_abs_shap = np.abs(shap_values_sev).mean(axis=0)

    mean_abs_sh

except Exception as e:
    print(f"\nSHAP analysis skipped due to environment constraints or error: {e}")

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
